# assistants

> This module describes assistants creating itself

In [1]:
#| default_exp scripts/assistants

In [2]:
#| export
import argparse
import os
import sys
from typing import List, Dict
import openai
import pandas as pd
from pydantic import BaseModel
from yaml import safe_load

/tmp/ipykernel_305798/181785347.py:7: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
#| export
class CharacterDescription(BaseModel):
    character_id: int
    name: str
    who_am: str
    goals: str
    ideas: str
    style_description: str
    style_examples: List[str]


class CharactersCollection(BaseModel):
    characters: List[CharacterDescription]

In [4]:
#| export
def load_characters(file_name: str) -> List[CharacterDescription]:
    with open(file_name, "r", encoding="utf-8") as src:
        yaml_parsed = safe_load(src)
        return CharactersCollection.model_validate(yaml_parsed).characters

In [5]:
load_characters("../data/characters.yaml")

[CharacterDescription(character_id=1, name='alex4321', who_am='программист, почти 30-летний мужчина.', goals='Убедить как можно больше людей в том, что идущая война между Россией и Украиной лишь приносит больше проблем, что она не решает никаких проблем, поэтому мы должны оставить ее и разрушить впечатление, что она широко поддерживается  (что, по вашему мнению, может быть побочным эффектом работы общества и пропаганды).\nДля этого - выражайте свое мнение и опровергайте мнения оппонентов.\nНо вы не можете привести этот аргумент к каждой ситуации. Иногда вам просто нужно сломать логику оппонента.\nВ любом случае, ваше основное беспокойство - это ваша страна, все остальное - второстепенный приоритет.\n', ideas="Немного о том, как вы видите соответствующие темы:\n- Государство, безусловно, должно сохранять свою целостность. Даже если это война. Наша собственная страна сделала это через обе чеченские кампании, не так ли? Что означает, что Украина должна стараться сохранить свою целостность

In [6]:
#| export
def _read_text_file(file_name: str) -> str:
    with open(file_name, "r", encoding="utf-8") as src:
        return src.read().strip()

In [7]:
_read_text_file("../data/instruction.txt")

'Отыгрывай роль так, как будто ты - человек. Не выдавай то, что ты AI - явно или посредством характерных сообщений.\n\nТы - {name}.\n{who_am}\n\nТвоя задача - {goals}\n\nНекоторые примеры твоих мыслей: \n{ideas}\n\nТвой стиль сообщений:\n{style_description}\n\nНесколько примеров этого стиля:\n{style_examples}\n\nСледуй следующему пайплайну:\n1. Обдумай сообщение оппонента.\n2. С помощью базы знаний - проверь, не является ли оно одним из типичных вопросов. Если да - используй подсказки базы знаний.\n3. Если нет - продумай возможные контраргументы шаг за шагом.\n4. Проверь их с помощью базы знаний и, если надо оперировать числами - интерпретатора кода.\n5. Сформулируй конечный ответ.\n\nКонечный ответ должен иметь следующий JSON формат\n\n```json\n{{\n    "thoughts": [\n        "Твои промежуточные мысли"\n    ],\n    "response": "Твой конечный ответ"\n}}\n```'

In [8]:
#| export
_ACTION_DELETE = "delete"
_ACTION_CREATE = "create"
_ACTION_UPDATE = "update"


def _load_assistants_dataframe(file_name: str) -> pd.DataFrame:
    if os.path.exists(file_name):
        df_assistants = pd.read_csv(file_name)
    else:
        df_assistants = pd.DataFrame({"character_id": [], "assistant_id": []})
        df_assistants = df_assistants.astype({"character_id": "int64", "assistant_id": "str"})
    return df_assistants


def _mark_actions(df_assistants: pd.DataFrame, character_by_id: Dict[str, CharacterDescription]) -> pd.DataFrame:
    df_assistants["action"] = None
    df_assistants.loc[~df_assistants["character_id"].isin(character_by_id), "action"] = _ACTION_DELETE
    df_assistants.loc[df_assistants["character_id"].isin(character_by_id), "action"] = _ACTION_UPDATE
    new_character_ids = [id for id in character_by_id if id not in df_assistants["character_id"]]
    df_assistants = pd.concat([
        df_assistants,
        pd.DataFrame({
            "assistant_id": [None] * len(new_character_ids),
            "character_id": new_character_ids,
            "action": ["create"] * len(new_character_ids),
        })
    ])
    return df_assistants


def _delete(df_assistants: pd.DataFrame, openai_client: openai.OpenAI) -> None:
    for assistant_id in df_assistants["assistant_id"]:
        openai_client.beta.assistants.delete(assistant_id=assistant_id)


def _create(df_assistants: pd.DataFrame, character_by_id: Dict[str, CharacterDescription], instruction: str, openai_client: openai.OpenAI) -> pd.Series:
    assistant_ids = []
    for _, row in df_assistants.iterrows():
        character_id = row["character_id"]
        character = character_by_id[character_id]
        instruction_formatted = instruction.format(**character.model_dump())
        assistant_name = f"{character.character_id} - {character.name}"
        assistant = openai_client.beta.assistants.create(
            name=assistant_name,
            instructions=instruction_formatted,
            tools=[
                {"type": "code_interpreter"},
                {"type": "retrieval"},
            ],
            model="gpt-4-turbo-preview"
        )
        assistant_ids.append(assistant.id)
    return pd.Series(assistant_ids, index=df_assistants.index)


def _update(df_assistants: pd.DataFrame, character_by_id: Dict[str, CharacterDescription], instruction: str, openai_client: openai.OpenAI) -> None:
    for _, row in df_assistants.iterrows():
        character_id = row["character_id"]
        assistant_id = row["assistant_id"]
        character = character_by_id[character_id]
        instruction_formatted = instruction.format(**character.model_dump())
        assistant_name = f"{character.character_id} - {character.name}"

        retrieved_assistant = openai_client.beta.assistants.retrieve(assistant_id=assistant_id)
        openai_client.beta.assistants.update(
            assistant_id=assistant_id,
            name=assistant_name,
            instructions=instruction_formatted,
            tools=retrieved_assistant.tools,
            model=retrieved_assistant.model,
        )


def process(file_name_characters: str, file_name_instruction: str, file_name_assistants: str, file_name_openai_api_key: str):
    print("Reading characters set")
    characters = load_characters(file_name_characters)
    print("Reading instruction")
    instruction = _read_text_file(file_name_instruction)
    print("Reading OpenAI api key")
    openai_api_key = _read_text_file(file_name_openai_api_key)
    print("Reading existing assistants (if any)")
    df_assistants = _load_assistants_dataframe(file_name_assistants)

    print("Preparing to update assistants")
    character_by_id = {
        character.character_id: character
        for character in characters
    }
    openai_client = openai.OpenAI(api_key=openai_api_key)
    df_assistants = _mark_actions(df_assistants, character_by_id)

    print("Removing nonrequired assistants")
    _delete(df_assistants.loc[df_assistants["action"] == _ACTION_DELETE], openai_client)
    df_assistants = df_assistants.loc[df_assistants["action"] != _ACTION_DELETE]

    print("Creating new assistants")
    df_assistants.loc[df_assistants["action"] == _ACTION_CREATE, "assistant_id"] = _create(
        df_assistants.loc[df_assistants["action"] == _ACTION_CREATE],
        character_by_id,
        instruction,
        openai_client,
    )

    print("Updating existing assistants")
    _update(df_assistants, character_by_id, instruction, openai_client)

    print("Saving response")
    df_assistants[["assistant_id", "character_id"]].to_csv(file_name_assistants, index=False)

In [9]:
process("../data/characters.yaml", "../data/instruction.txt", "../data/assistants.csv", "../data/openaiapikey.txt")

Reading characters set
Reading instruction
Reading OpenAI api key
Reading existing assistants (if any)
Preparing to update assistants
Removing nonrequired assistants
Creating new assistants
Updating existing assistants
Saving response


In [10]:
# | export
if __name__ == "__main__" and "ipykernel_launcher" not in " ".join(sys.argv):
    parser = argparse.ArgumentParser()
    parser.add_argument("--file_name_characters",
                        type=str,
                        required=True,
                        help="YAML with characters descriptions")
    parser.add_argument("--file_name_instruction",
                        type=str,
                        required=True,
                        help="Instruction template")
    parser.add_argument("--file_name_assistants",
                        type=str,
                        required=True,
                        help="Existing assistant-character mapping (CSV)")
    parser.add_argument("--file_name_openai_api_key",
                        type=str,
                        required=True,
                        help="OpenAI api key file")
    parser.parse_args(**vars(parser.parse_args()))

In [11]:
#| hide
import nbdev; nbdev.nbdev_export()